In [ ]:
%matplotlib inline
import utils; reload(utils)
from utils import *
import re
from __future__ import division, print_function

In [ ]:
import pandas
titanic_csv = pandas.read_csv('data/titanic/train.csv')
titanic_csv

In [ ]:
adjusted_titanic_csv = titanic_csv.copy()
average_age = np.round((np.sum(adjusted_titanic_csv['Age'])/len(adjusted_titanic_csv['Age'])))
print ("average_age:",average_age)
print ("average_survival", np.sum(adjusted_titanic_csv['Survived'])/len(adjusted_titanic_csv['Survived']))

for i in xrange(0, len(adjusted_titanic_csv['Age'])):
    if np.isnan(adjusted_titanic_csv.at[i,'Age']):
        adjusted_titanic_csv.at[i,'Age'] = average_age
        
adjusted_titanic_csv = adjusted_titanic_csv[adjusted_titanic_csv.columns[[2,4,5,6,7,9,11,1]]]
adjusted_titanic_csv['Sex'] = map(lambda x: 1 if x=='male' else 0, adjusted_titanic_csv['Sex'])

embarkedConversion = {
    'S': 0,
    'C': 1,
    'Q': 2,
    'nan' : 3,
}
adjusted_titanic_csv['Embarked'] = map(lambda x: embarkedConversion[x] if type(x)==str else 3, adjusted_titanic_csv['Embarked'])
adjusted_titanic_csv

In [ ]:
import random
random.seed(42)
split = int(0.9*len(adjusted_titanic_csv))
X_cols = adjusted_titanic_csv.columns[:-1]
Y_cols = adjusted_titanic_csv.columns[-1:]
X_train = adjusted_titanic_csv[X_cols][:split]
Y_train = adjusted_titanic_csv[Y_cols][:split]
X_test = adjusted_titanic_csv[X_cols][split:]
Y_test = adjusted_titanic_csv[Y_cols][split:]

X_train.describe().columns,Y_train.describe().columns, X_train.shape, Y_train.shape

In [ ]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

In [ ]:
from keras.layers.advanced_activations import LeakyReLU, PReLU
model = Sequential()
model.add(keras.layers.Dense(1024,input_shape=X_train.shape[1:], activation='linear'))
model.add(LeakyReLU(alpha=.001))
model.add(keras.layers.Dropout(0.35))
model.add(keras.layers.Dense(1024, activation='linear'))
model.add(LeakyReLU(alpha=.001))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(2, activation="softmax"))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.summary(), model.optimizer.lr.get_value()

In [ ]:
model.optimizer.lr.set_value(0.0001)

In [ ]:
model.fit(X_train, Y_train, nb_epoch=500, batch_size=1024, validation_data=(X_test, Y_test), verbose=2)

In [ ]:
titanic_csv_test = pandas.read_csv('data/titanic/test.csv')
titanic_csv_test

In [ ]:
adjusted_titanic_csv_test = titanic_csv_test.copy()

for i in xrange(0, len(adjusted_titanic_csv_test['Age'])):
    if np.isnan(adjusted_titanic_csv_test.at[i,'Age']):
        adjusted_titanic_csv_test.at[i,'Age'] = average_age
        
adjusted_titanic_csv_test = adjusted_titanic_csv_test[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
adjusted_titanic_csv_test['Sex'] = map(lambda x: 1 if x=='male' else 0, adjusted_titanic_csv_test['Sex'])

adjusted_titanic_csv_test['Embarked'] = map(lambda x: embarkedConversion[x] if type(x)==str else 3, adjusted_titanic_csv_test['Embarked'])
adjusted_titanic_csv_test[0:10]

In [ ]:
X_input = np.array(adjusted_titanic_csv_test)
X_input.shape

In [ ]:
Y_output = model.predict(X_input)
Y_output.shape, Y_output[:20]

In [ ]:
survivers = map(lambda x: 0 if x[0]>0.35 else 1, Y_output)
survivers[:20], np.sum(survivers)/len(survivers)

In [ ]:
import time
output = pd.DataFrame(columns=['PassengerId','Survived'])
for i in xrange(len(survivers)):
    output.loc[i] = (892+i, survivers[i])
name = 'data/titanic/'+str(time.time())+'_dbuschho.csv'
output.to_csv(name,index=False)
print (name)
output